In [2]:
import pandas as pd

# Load the dataset
file_path = '/mnt/data/Project_2.O.csv'
data = pd.read_csv('C:/Users/prano/Downloads/Project_2.O.csv')

# Convert the total battery capacity from kW to kWh
# Assuming 1 hour of supply time, the conversion is direct
total_battery_capacity_kwh = 9.73228  # This is the battery capacity in kWh

# Use columns from the dataset
soc = data['Pack SoC']  # State of Charge in percentage
voltage = data['Voltage']  # Voltage in volts
input_power = data['input_power']  # Input power in kW (assumed)

# Typical values for calculations
energy_consumption_rate_kwh_per_km = 0.2  # kWh per kilometer (adjust as necessary)
average_speed_kmph = 60  # Assuming an average speed of 60 km/h

# Step 1: Calculate remaining energy in kWh
remaining_energy_kwh = (soc / 100) * total_battery_capacity_kwh

# Step 2: Calculate remaining range (in kilometers) based on energy consumption rate
remaining_range_km = remaining_energy_kwh / energy_consumption_rate_kwh_per_km

# Step 3: Calculate the remaining time in hours (distance / speed)
remaining_time_hours = remaining_range_km / average_speed_kmph

# Step 4: Convert remaining time from hours to minutes
remaining_time_minutes = remaining_time_hours * 60

# Add the calculated remaining time in minutes to the dataset
data['Remaining Time (minutes)'] = remaining_time_minutes

# Display the first few rows with the new column showing calculated remaining time
output = data[['Pack SoC', 'Voltage', 'input_power', 'Remaining Time (minutes)']]
print(output.head())


   Pack SoC  Voltage  input_power  Remaining Time (minutes)
0      94.0     57.3      0.00573                 45.741716
1      94.0     57.3      0.00573                 45.741716
2      94.0     57.3      0.00573                 45.741716
3      94.0     57.3      0.00573                 45.741716
4      94.0     57.3      0.00573                 45.741716


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Load the dataset
data = pd.read_csv('C:/Users/prano/Downloads/Project_2.O.csv')

# Use columns from the dataset (Pack SoC, Voltage, input_power) to predict Remaining Time
X = data[['Pack SoC', 'Voltage', 'input_power']]  # Input features

# Total battery capacity in kWh (as provided earlier)
total_battery_capacity_kwh = 9.73228
energy_consumption_rate_kwh_per_km = 0.2  # kWh per kilometer
average_speed_kmph = 60  # Average speed in km/h

# Calculate remaining energy in kWh
remaining_energy_kwh = (data['Pack SoC'] / 100) * total_battery_capacity_kwh

# Calculate remaining range in kilometers
remaining_range_km = remaining_energy_kwh / energy_consumption_rate_kwh_per_km

# Calculate the remaining time in hours (distance / speed)
remaining_time_hours = remaining_range_km / average_speed_kmph

# Convert remaining time to minutes
data['Remaining Time (minutes)'] = remaining_time_hours * 60

# Now use this calculated column as the target variable (y)
y = data['Remaining Time (minutes)']  # Output label (target)

# Step 1: Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 2: Scale the features for better performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 3: Build the ANN model
model = Sequential()

# Input Layer + Hidden Layer 1 (with 32 neurons and ReLU activation)
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden Layer 2 (with 16 neurons and ReLU activation)
model.add(Dense(16, activation='relu'))

# Output Layer (1 neuron, predicting the remaining time in minutes)
model.add(Dense(1))

# Step 4: Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 5: Early Stopping to avoid overfitting (patience of 10 epochs)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Step 6: Train the model using the training data with early stopping
history = model.fit(X_train_scaled, y_train, 
                    validation_split=0.2, 
                    epochs=200,  # Set a large number of epochs
                    callbacks=[early_stopping], 
                    verbose=1)

# Step 7: Evaluate the model using the test data
test_loss = model.evaluate(X_test_scaled, y_test)

# Step 8: Make predictions on the test data
y_pred = model.predict(X_test_scaled)

# Convert predictions to a 1D array
y_pred = y_pred.flatten()

# Step 9: Calculate regression accuracy metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Step 10: Print overall loss values after training
final_training_loss = history.history['loss'][-1]
final_validation_loss = history.history['val_loss'][-1]
print(f'Final Training Loss: {final_training_loss}')
print(f'Final Validation Loss: {final_validation_loss}')

# Display the metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')

# Step 11: Print actual vs predicted values for test data
print('Actual vs Predicted Remaining Time (minutes):')
for actual, predicted in zip(y_test[:10], y_pred[:10]):  # Display first 10 predictions
    print(f'Actual: {actual:.2f}, Predicted: {predicted:.2f}')


final_training_loss = history.history['loss'][-1]
final_validation_loss = history.history['val_loss'][-1]
print(f'Final Training Loss: {final_training_loss}')
print(f'Final Validation Loss: {final_validation_loss}')

# Step 12: Take user input for prediction
def predict_remaining_time():
    try:
        # Take user input for Pack SoC, Voltage, and input_power
        pack_soc = float(input("Enter the Pack SoC (%): "))
        voltage = float(input("Enter the Voltage (V): "))
        input_power = float(input("Enter the Input Power (kW): "))

        # Prepare the user input as a single-row DataFrame
        user_input = pd.DataFrame([[pack_soc, voltage, input_power]], 
                                  columns=['Pack SoC', 'Voltage', 'input_power'])

        # Scale the input using the same scaler used on training data
        user_input_scaled = scaler.transform(user_input)

        # Predict the remaining time
        predicted_time = model.predict(user_input_scaled)

        # Output the predicted time in minutes
        print(f'Predicted Remaining Time: {predicted_time[0][0]:.2f} minutes')
    except ValueError:
        print("Invalid input. Please enter numeric values.")

# Call the function to take input from the user and predict
predict_remaining_time()




Epoch 1/200


C:\Users\prano\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1762.3372 - val_loss: 1647.9974
Epoch 2/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1614.3674 - val_loss: 1393.5587
Epoch 3/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1303.1487 - val_loss: 939.6408
Epoch 4/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 821.2204 - val_loss: 474.5025
Epoch 5/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 401.0673 - val_loss: 240.9066
Epoch 6/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 232.6552 - val_loss: 172.3837
Epoch 7/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 170.8234 - val_loss: 130.4257
Epoch 8/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 128.9927 - val_loss: 95.2506
Epoch 9/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 90.7882 - val_loss: 68.2775
Epoch 10/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 63.5293 - val_loss: 49.7774
Epoch 11/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41.9378 - val_loss: 36.3813
Epoch 12/200
50/50 ━━━━━━━━━━━━━━

Enter the Pack SoC (%):  97
Enter the Voltage (V):  52
Enter the Input Power (kW):  9.7


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted Remaining Time: 51.57 minutes


In [17]:
# Create a new DataFrame with the features and actual/predicted values
output_data = pd.DataFrame({
    'Pack SoC': X_test['Pack SoC'].values,
    'Voltage': X_test['Voltage'].values,
    'input_power': X_test['input_power'].values,
    'Actual Remaining Time (minutes)': y_test.values,
    'Predicted Remaining Time (minutes)': y_pred
})

# Save the dataset to a CSV file
output_data.to_csv('predicted_vs_actual.csv', index=False)

